In [65]:
import pandas as pd

import numpy as np

import time

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score

from sklearn.model_selection import KFold, train_test_split

from sklearn.utils import shuffle

In [66]:
t0 = time.time()
df = pd.read_csv('aggregate-20160501.csv')
t1 = time.time()
print(str((t1-t0)/60) + " minutes")

1.1862463196118673 minutes


In [67]:
df.head()

,article_name,num_edits,views_30d,views_7d,views_3d,views_1d,edits_30d,edits_7d,edits_3d,edits_1d,...,talk_minor_edits_30d,talk_minor_edits_7d,talk_minor_edits_3d,talk_minor_edits_1d,talk_avg_size_30d,talk_avg_size_7d,talk_avg_size_3d,talk_avg_size_1d,talk_avg_size,talk_latest_size
0,!,0.0,829.0,133.0,75.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,14441.666667,NaN,8470.0,0.0,10359.000000,0.0
1,!!!,0.0,3482.0,641.0,267.0,124.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,26219.071429,28604.0,56698.0,14324.0,27452.529412,14324.0
2,!!!Fuck_You!!!_and_Then_Some,0.0,391.0,53.0,41.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,!!!_(Chk_Chk_Chk),0.0,86.0,22.0,8.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,!!!_(album),0.0,628.0,91.0,47.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13793.250000,8771.0,8788.0,8787.0,12790.450000,8787.0


In [68]:
df.shape

(12220462, 38)

In [69]:
for c in df.columns:
    print(c)

article_name
num_edits
views_30d
views_7d
views_3d
views_1d
edits_30d
edits_7d
edits_3d
edits_1d
minor_edits_30d
minor_edits_7d
minor_edits_3d
minor_edits_1d
avg_size_30d
avg_size_7d
avg_size_3d
avg_size_1d
avg_size
latest_size
talk_views_30d
talk_views_7d
talk_views_3d
talk_views_1d
talk_edits_30d
talk_edits_7d
talk_edits_3d
talk_edits_1d
talk_minor_edits_30d
talk_minor_edits_7d
talk_minor_edits_3d
talk_minor_edits_1d
talk_avg_size_30d
talk_avg_size_7d
talk_avg_size_3d
talk_avg_size_1d
talk_avg_size
talk_latest_size


In [70]:
set(df.num_edits)

{0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 25.0,
 26.0,
 27.0,
 29.0,
 30.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 40.0,
 42.0,
 47.0,
 49.0,
 52.0,
 57.0,
 60.0,
 62.0,
 67.0,
 77.0}

In [71]:
len(df["num_edits"][df["num_edits"] == 0])/len(df["num_edits"])*100

99.94655684866906

In [72]:
df['num_edits_binary'] = df['num_edits'].apply(lambda x: int(x > 0))
set(df['num_edits_binary'])

{0, 1}

In [73]:
print("Missing Observations \n")

print(df.isnull().sum())

Missing Observations 

article_name                   0
num_edits                      0
views_30d                      0
views_7d                       0
views_3d                       0
views_1d                       0
edits_30d                      0
edits_7d                       0
edits_3d                       0
edits_1d                       0
minor_edits_30d                0
minor_edits_7d                 0
minor_edits_3d                 0
minor_edits_1d                 0
avg_size_30d             1564532
avg_size_7d              9642891
avg_size_3d             10546205
avg_size_1d             10952764
avg_size                   20394
latest_size                20394
talk_views_30d                 0
talk_views_7d                  0
talk_views_3d                  0
talk_views_1d                  0
talk_edits_30d                 0
talk_edits_7d                  0
talk_edits_3d                  0
talk_edits_1d                  0
talk_minor_edits_30d           0
talk_minor_edits_7d 

In [74]:
feature_names = ["views_30d", 
                 "edits_30d", 
                 "minor_edits_30d", 
                 "talk_views_30d",
                 "talk_edits_30d",
                 "talk_minor_edits_30d"]

label_name = "num_edits_binary"

X = df[feature_names].as_matrix()

Y = df[label_name].as_matrix()

In [75]:
zero = len(df["num_edits"][df["num_edits"] == 0])/len(df["num_edits"])
one = len(df["num_edits"][df["num_edits"] != 0])/len(df["num_edits"])

In [76]:
# balanced sample

np.random.seed(seed=13579)

set1_idx = np.random.choice(range(len(df)), int(len(df) * .5), replace=False)
set2_idx = list(set(range(len(df))) - set(list(set1_idx)))

In [77]:
set1_X = df.loc[set1_idx, feature_names]
set1_Y = df.loc[set1_idx, label_name]

In [78]:
set2 = df.loc[set2_idx, list(df.columns.tolist())]

In [79]:
edited = set2[set2.num_edits > 0.0].copy(deep=True)
edited.shape

(3270, 39)

In [80]:
not_edited = set2[set2.num_edits == 0.0].copy(deep=True)

In [81]:
not_edited_selected = not_edited[0:3270]

In [82]:
balanced_set = pd.concat([edited, not_edited_selected])

In [83]:
balanced_set = shuffle(balanced_set)

In [84]:
balanced_set.shape

(6540, 39)

In [85]:
# logistic regression

X_train, X_test, y_train, y_test = train_test_split(balanced_set[feature_names], balanced_set[label_name],
                                                    train_size=0.66, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [86]:
print(len(X_test))
print(len(set1_X))

2159
6110231


In [87]:
zeros = np.array([0]*2159)
zeros_i = np.array([0]*6110231)

ones = np.array([1]*2159)
ones_i = np.array([1]*6110231)

In [88]:
y_pred = model.predict(X_test)
y_pred.shape

(2159,)

In [89]:
y_pred_i = model.predict(set1_X)
y_pred_i.shape

(6110231,)

In [90]:
print(model.score(X_test, y_test))
print(model.score(set1_X, set1_Y))

0.662806855025
0.991236665193


In [91]:
print(model.score(set1_X, zeros_i))

0.991304256746


In [92]:
print(precision_recall_fscore_support(y_test, y_pred))
print(precision_recall_fscore_support(set1_Y, y_pred_i))

(array([ 0.61695138,  0.77237049]), array([ 0.86623616,  0.45767442]), array([ 0.72064467,  0.57476636]), array([1084, 1075]))
(array([ 0.99969672,  0.02680067]), array([ 0.99153279,  0.43667587]), array([ 0.99559802,  0.05050183]), array([6106970,    3261]))


In [93]:
print(precision_recall_fscore_support(set1_Y, zeros_i))

(array([ 0.9994663,  0.       ]), array([ 1.,  0.]), array([ 0.99973308,  0.        ]), array([6106970,    3261]))


/Users/ugur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [94]:
print(f1_score(y_test, y_pred))
print(f1_score(set1_Y, y_pred_i))

0.57476635514
0.0505018264354


In [95]:
print(f1_score(set1_Y, zeros_i))

0.0


/Users/ugur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [96]:
np.random.seed(seed=13579)

kf = KFold(n_splits=2, random_state=13579)

train_acc = []
test_acc = []

train_precision_recall_fscore_support = []
test_precision_recall_fscore_support = []

train_acc_weighted = []
test_acc_weighted = []

t0 = time.time()
for train_idx, test_idx in kf.split(X):
    train_x = X[train_idx]
    train_y = Y[train_idx]
    test_x = X[test_idx]
    test_y = Y[test_idx]

    model = LogisticRegression()
    model.fit(train_x, train_y)
    
    train_acc += [accuracy_score(train_y, model.predict(train_x))]
    test_acc += [accuracy_score(test_y, model.predict(test_x))]
    
    train_precision_recall_fscore_support += [precision_recall_fscore_support(train_y, 
                                                                              model.predict(train_x),
                                                                              average='binary')]
    test_precision_recall_fscore_support += [precision_recall_fscore_support(test_y, 
                                                                             model.predict(test_x),
                                                                             average='binary')]
    
    model = LogisticRegression(class_weight = {0: one, 1: zero})
    model.fit(train_x, train_y)
    
    train_acc_weighted = [accuracy_score(train_y, model.predict(train_x))]
    test_acc_weighted = [accuracy_score(test_y, model.predict(test_x))]
    
t1 = time.time()
print(str((t1-t0)/60) + " minutes")

print(np.mean(train_acc))
print(np.mean(test_acc))

print(train_precision_recall_fscore_support)
print(test_precision_recall_fscore_support)

print(np.mean(train_acc_weighted))
print(np.mean(test_acc_weighted))

1.7166828513145447 minutes
0.99945640353
0.999463113588
[(0.75, 0.016949152542372881, 0.033149171270718238, None), (0.37608695652173912, 0.027226943657538558, 0.050777810390372755, None)]
[(0.43965517241379309, 0.016052880075542966, 0.030974795019738842, None), (0.375, 0.016949152542372881, 0.032432432432432434, None)]
0.909478054103
0.999489217347


In [21]:
# decision tree

In [22]:
# linear regression

In [23]:
# k-nearest neighbors